<a href="https://colab.research.google.com/github/EclipticWin/Data-Analysis-with-Open-Source/blob/main/%EC%98%A4%ED%94%88%EC%86%8C%EC%8A%A4_%EB%8D%B0%EC%9D%B4%ED%84%B0_%EB%B6%84%EC%84%9D_4%EA%B0%95.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# 오픈소스 기반 데이터 분석 4강 - 데이터 수집


## 4-1 CSV 파일 읽기

In [3]:
# 우와 새로운 분이다

import pandas as pd

## data.csv 파일 읽기
df = pd.read_csv('data.csv', encoding='utf-8', sep=',', header=0,
                 index_col=None, skiprows=None, nrows=None)

# 파일 이름, 인코딩(중요), sep - 구분자
# header - 스키마가 첫줄에 있는 경우 대비(여러 줄의 헤더 있을 경우 값 다시 1,2 등으로 변경)
# skiprows - 불필요 행 뛰어넘고 읽기
# 이런 옵션들은 선택적이지만 정상적으로 읽히지 않을 경우 사용할 줄 알아야 함

print(df)

           날짜    체중  골격근량  체지방량
0  2025.02.06  64.7  30.0  11.1
1  2025.02.04  64.0  29.3  11.6


## 4-2 JSON 파일 읽기



In [6]:
import json
import pandas as pd

## data.json 파일 출력
with open('data.json', mode='r', encoding='utf-8') as f:
  data = json.load(f)
print(data)
## data.json 파일 DataFrame 읽기
df = pd.read_json('data.json', orient='records', encoding='utf-8', lines=False)
print(df)

{'매출데이터': [{'월': '2025-01', '매출액': 1000000, '비용': 700000, '이익': 300000}, {'월': '2025-02', '매출액': 1200000, '비용': 800000, '이익': 400000}, {'월': '2025-03', '매출액': 1500000, '비용': 900000, '이익': 600000}]}
                                               매출데이터
0  {'월': '2025-01', '매출액': 1000000, '비용': 700000,...
1  {'월': '2025-02', '매출액': 1200000, '비용': 800000,...
2  {'월': '2025-03', '매출액': 1500000, '비용': 900000,...


## 4-3 텍스트 파일 읽기 및 데이터 추출

In [8]:
# 주민번호 패턴 검출 후 마스킹해서 다시 저장하는 과정
import re

## 파일(callcenter20250301.log) 오픈 및 읽기
with open('callcenter20250301.log', 'r', encoding='utf-8') as f:
    content = f.read()

## 주민등록번호 패턴 생성
# 정규식 만들기
pattern = re.compile(r'(\d{6})-(\d{7})')

## 주민등록번호 마스킹
masked_content = pattern.sub(r'\1-*******', content)

## 마스킹된 파일(callcenter20250301_masked.log) 오픈 및 쓰기
with open('callcenter20250301_masked.log', mode='w') as f:
    f.write(masked_content)

print("주민등록번호 마스킹 완료. 'callcenter20250301_masked.log.txt' 파일로 저장되었습니다.")

주민등록번호 마스킹 완료. 'callcenter20250301_masked.log.txt' 파일로 저장되었습니다.


## 4-4 Open-Meteo의 무료 날씨 API를 통한 특정 지역 온도 조회

In [11]:
import requests
import json

url = "https://api.open-meteo.com/v1/forecast?=&=&current=temperature_2m"
params = {
    "latitude": "37.58638333",
    "longitude": "127.0203333",
    "current": "temperature_2m"
}

try:
    ## URL 및 파라미터 전송
    response = requests.get(url, params=params)
    response.raise_for_status()

    ## JSON 데이터 읽기
    data = response.json()

    print("API 응답:", data)
    print("서울시 종로구의 현재 온도는 : {0}{1} 입니다.".format(data['current']['temperature_2m'], data['current_units']['temperature_2m']))

except requests.exceptions.RequestException as e:
    print(f"API 호출 실패: {e}")
except json.JSONDecodeError as e:
    print(f"JSON 파싱 실패: {e}")

API 응답: {'latitude': 37.6, 'longitude': 127.0, 'generationtime_ms': 0.02467632293701172, 'utc_offset_seconds': 0, 'timezone': 'GMT', 'timezone_abbreviation': 'GMT', 'elevation': 29.0, 'current_units': {'time': 'iso8601', 'interval': 'seconds', 'temperature_2m': '°C'}, 'current': {'time': '2025-10-12T05:15', 'interval': 900, 'temperature_2m': 21.3}}
서울시 종로구의 현재 온도는 : 21.3°C 입니다.


## 4-5 Selenium과 lxml을 이용한 웹 스크래핑

In [ ]:
!curl -o google-chrome-stable_current_amd64.deb https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!apt install ./google-chrome-stable_current_amd64.deb -y
!pip install selenium webdriver_manager

In [18]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from lxml import html
import time

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')               # 브라우저 창 없이 실행 (headless 이 모드 알아두기)
chrome_options.add_argument('--no-sandbox')             # 보안모드 비활성화 (Colab 필수)
chrome_options.add_argument('--disable-dev-shm-usage')  # 메모리 부족 방지 (Colab 필수)
chrome_options.add_argument('--window-size=1920x1080')  # 창 크기 설정(가상)
chrome_options.add_argument('--disable-gpu')            # GPU 가속 비활성화 (일부 환경 안정성)
chrome_options.binary_location = "/usr/bin/google-chrome-stable"  # Colab용 크롬 경로 지정

## 드라이버 실행
driver = webdriver.Chrome(options=chrome_options)

## 사이트 접속
url = 'https://professor.knou.ac.kr/jaehwachung/index.do'
driver.get(url)


## 사이트 접속 대기
time.sleep(2)

# print(driver.page_source)


# 첫 로딩 실패 방지용 - 다시 한 번 로딩 => 이거 있어야 제대로 결과값 뜸... 보안인가??
driver.get(url)
time.sleep(2)

## 페이지 제목 출력
page_source = driver.page_source
tree = html.fromstring(page_source)

title_text = tree.xpath('//title/text()')
print(title_text)

## 드라이버 종료
driver.quit()

['컴퓨터과학과 정재화 교수']



# 실습 시나리오

## 공공데이터 포털 가입 및 데이터 신청

- [https://www.data.go.kr](https://www.data.go.kr)
- 한국환경공단 에어코리아 대기오염정보 데이터 신청

In [36]:
import requests

## 데이터 수집 url 및 api key 설정
# 251012 공공데이터포털 화제로 인해 실습 불가
# url = '시도별 실시간 측정정보 조회 -> 조회하고 요청주소 여기에 복붙'
# api_key = '마이페이지에 있는 개인 api인증키 Decording Key 복붙'

# params = {
#     'serviceKey': api_key,
#     'returnType': 'json',
#     'numOfRows': '100',
#     'pageNo': '1',
#     'sidoName': '서울',
#     'ver': '1.0'
# }

# ## 데이터 수집
# response = requests.get(url, params=params)

# ## 호출 성공/실패 출력
# print(response.json())




# 서울 열린데이터 광장 다른 데이터로 바꿔서 해봄
api_key = '내api키'
gu_name = '종로구'
url = f'http://openapi.seoul.go.kr:8088/{api_key}/json/ListRainfallService/1/5/{gu_name}'

## 데이터 수집
response = requests.get(url)

## 호출 성공/실패 출력
print(response.json())

print("응답 상태 코드:", response.status_code)


{'ListRainfallService': {'list_total_count': 11914, 'RESULT': {'CODE': 'INFO-000', 'MESSAGE': '정상 처리되었습니다'}, 'row': [{'RAINGAUGE_CODE': 1002.0, 'RAINGAUGE_NAME': '부암동', 'GU_CODE': 110.0, 'GU_NAME': '종로구', 'RAINFALL10': '0', 'RECEIVE_TIME': '2025-10-12 14:49'}, {'RAINGAUGE_CODE': 1001.0, 'RAINGAUGE_NAME': '종로구청', 'GU_CODE': 110.0, 'GU_NAME': '종로구', 'RAINFALL10': '0', 'RECEIVE_TIME': '2025-10-12 14:49'}, {'RAINGAUGE_CODE': 1001.0, 'RAINGAUGE_NAME': '종로구청', 'GU_CODE': 110.0, 'GU_NAME': '종로구', 'RAINFALL10': '0', 'RECEIVE_TIME': '2025-10-12 14:39'}, {'RAINGAUGE_CODE': 1002.0, 'RAINGAUGE_NAME': '부암동', 'GU_CODE': 110.0, 'GU_NAME': '종로구', 'RAINFALL10': '0', 'RECEIVE_TIME': '2025-10-12 14:39'}, {'RAINGAUGE_CODE': 1002.0, 'RAINGAUGE_NAME': '부암동', 'GU_CODE': 110.0, 'GU_NAME': '종로구', 'RAINFALL10': '0', 'RECEIVE_TIME': '2025-10-12 14:29'}]}}
응답 상태 코드: 200
